In [ ]:
%matplotlib ipympl

import numpy as np
import matplotlib.pyplot as plt
import pyvisa

from E5063A import *

In [ ]:
# List the resources connected
rm = pyvisa.ResourceManager()
rm.list_resources()

In [ ]:
# Parameters
vna_address = 'GPIB0::17::INSTR' # Address to VNA
                                 # Run cell above to find address if needed

save_data = 1
save_file_name = 'test.csv'

if_bandwidth = 1e3  # Hz, IF-Bandwidth

start_freq = 3e9 # Hz, Start Frequency
stop_freq = 6e9 # Hz, Stop Frequency
num_points = 15001 # Number of Points


s = 'S12' # S-parameter to measure
s_format = 'PLOG' # Data Format
                  # PLOG for log magnitude and phase
                  # PLin for lin magnitude and phase
                  # MLOG for lin magnitude
                  # MLIN for linear magnetude

In [ ]:
# Connect to the VNA
rm = pyvisa.ResourceManager()
vna = rm.open_resource(vna_address)

# Query the VNA's identification
idn = vna.query('*IDN?')
print(f'VNA being used: {idn}')

vna.close()

In [ ]:
# Configure VNA
config_start_stop_freq(vna_address, start_freq, stop_freq)
config_vna(vna_address, s, s_format, num_points, if_bandwidth) 

# Get VNA configuration
vna_config = get_vna_config(vna_address)
print(f'Start Frequency: {vna_config[0]:.3f} Hz')
print(f'Stop Frequency: {vna_config[1]:.3f} Hz')
print(f'Number of Points: {vna_config[2]:.0f}')
print(f'IF Bandwidth: {vna_config[3]:.3f} Hz')
print(f'Power Level: {vna_config[4]:.3f} dB')

In [ ]:
# Perform measurement
config_s_format(vna_address, 'PLOG')
freq, s_mag, s_phase = measure_s(vna_address)
plot_s(freq, s_mag, s_phase)
autoscale_vna(vna_address)
config_s_format(vna_address, 'MLOG')

# Save measurement
if save_data == 1:
    save_s(freq, s_mag, s_phase, save_file_name)